In [1]:
import os
import fnmatch
from collections import defaultdict

answerDictionary = defaultdict(list)

entries = []

input_path = "C:/Users/milos/Desktop/PSML2023/Asteroid/public/inputs"


for root, dirs, files in os.walk(input_path):
     for file in files:
         #print(file)
        
         with open(os.path.join(root, file), "r",  encoding='utf-8-sig') as text:
                 entries.append(list(text))
                    
    
                    
                    

In [206]:
import numpy as np 
import math as m

line = entries[6]
coordinates = []
for el in line:
    a = tuple(int(x) for x in str(el).strip('()\n').split(','))
    coordinates.append(a)

def find_min_max(coordinates):
    x_arr = []
    y_arr = []
    z_arr = []
    
    for el in coordinates:
        x_arr.append(el[0])
        y_arr.append(el[1])
        z_arr.append(el[2])
    
    return ((min(x_arr), max(x_arr)),(min(y_arr), max(y_arr)),(min(z_arr), max(z_arr)))

bounds = find_min_max(coordinates)



dx, dy, dz = bounds[0][1] - bounds[0][0], bounds[1][1] - bounds[1][0], bounds[2][1] - bounds[2][0]

extra = 4
space = np.zeros((dx+extra+1, dy + 1 + extra, dz + extra +1))


for cube in coordinates:

    space[cube[0]-bounds[0][0] + 1:cube[0]+1-bounds[0][0] +1, 
          cube[1]-bounds[1][0] + 1:cube[1]-bounds[1][0]+1 +1,
          cube[2]-bounds[2][0] + 1:cube[2] -bounds[2][0]+1 + 1] = 1


def area(x, y, z, space):
    up =  space[x + 1, y, z] + space[x , y+1, z] + space[x , y, z+1] 
    down = space[x , y - 1, z] + space[x , y, z - 1] + space[x-1 , y, z]
    return 6 - up - down

total_area = 0
for x in range(0, dx+extra ):
    for y in range(0, dy +extra):
        for z in range(0, dz+extra):
            if space[x, y, z] :
                total_area += area(x, y, z, space)


def canVisit(space, row, col, visited, layer):
    global ROW, COL
    
    return ((row >= 0) and (row < ROW) and (col >= 0) and (col < COL) 
           and (space[row][col] and not visited[row][col])) and not layer[row][col]



def DFS(space, row, col, visited, count, layer):
    rowN = [-1, 0, 1, 0]
    colN = [0, -1, 0, 1]
    
    visited[row][col] = True
    
    for k in range(4):
        if(canVisit(space, row + rowN[k],
                     col + colN[k], visited, layer)):
            count[0] += 1
            DFS(space, row + rowN[k],
                     col + colN[k],visited, count, layer)
            
def largestRegion(space, layer):
    global ROW, COL
    
    visited = [[0] * COL for i in range(ROW)]
    ver = []
    result = -1
    
    for i in range(ROW):
        for j in range(COL):
 
            if (space[i][j] and not visited[i][j] and not layer[i][j]):
                 
                
                count = [1]
                DFS(space, i, j, visited, count, layer)

                if(count[0] > result):

                    result = count[0]

    return result
    
x_up = 0
x_down = 0
y_down = 0
y_up = 0
z_up = 0
z_down = 0

ROW = dx +extra
COL = dy +extra

max_top = -1
max_bot = -1
for z in range(0, dz +extra):
    top_layer = space[0:-1, 0:-1, z + 1]
    current_layer = space[0:-1, 0:-1, z]
    bottom_layer =  space[0:-1, 0:-1, z - 1]
    
    top_area = largestRegion(current_layer, top_layer)
    bottom_area = largestRegion(current_layer, bottom_layer)
    
    if top_area > max_top:
        max_top = top_area
        z_up = (top_area, ('z', z, 1))
    if bottom_area > max_bot:
        max_bot = bottom_area
        z_down = (bottom_area, ('z', z, -1))
    

ROW = dy +extra
COL = dz +extra
max_top = -1
max_bot = -1

for x in range(0, dx +extra):
    top_layer = space[x+1, 0:-1, 0:-1]
    current_layer = space[x, 0:-1, 0:-1]
    bottom_layer =  space[x-1, 0:-1, 0:-1]
    
    top_area= largestRegion(current_layer, top_layer)
    bottom_area = largestRegion(current_layer, bottom_layer)
    
    if top_area > max_top:
        max_top = top_area
        x_up = (top_area, ('x', x, 1))
    if bottom_area > max_bot:
        max_bot = bottom_area
        x_down = (bottom_area, ('x', x, -1))
        
ROW = dx +extra
COL = dz +extra

max_top = -1
max_bot = -1
for y in range(0, dy +extra):
    top_layer = space[0:-1, y+1, 0:-1]
    current_layer = space[0:-1, y, 0:-1]
    bottom_layer =  space[0:-1, y-1, 0:-1]
    
    top_area = largestRegion(current_layer, top_layer)
    bottom_area = largestRegion(current_layer, bottom_layer)
    
    if top_area > max_top:
        max_top = top_area
        y_up = (top_area, ('y', y, 1))
    if bottom_area > max_bot:
        max_bot = bottom_area
        y_down = (bottom_area, ('y', y, -1))

all_areas = [x_up, x_down, y_up, y_down, z_up, z_down]



def create(space):
    
    global Surface


    class Surface(object):
            _space = space #dictionary


            def __hash__(self):
                return hash(self.id)

            def __init__(self, coordinates):
                self.id = coordinates


                self.up = 0
                self.down = 0
                self.left = 0
                self.right = 0
                self.d1 = 0
                self.d2 = 0
                self.d3 = 0
                self.d4 = 0
                self.all = 0
                
                self.angle = 45

            
            def __repr__(self):
                return str(self.id)

            def find_surroudings(self):
                x = self.id[0]
                y = self.id[1]
                key = (x, y)
                
                decay = 0.95
                while self._space[key]:
                    self.up += decay
                    decay = decay*decay + 0.1
                    x+=1
                    key = (x, y)
                
                x = self.id[0]
                y = self.id[1]
                key = (x, y)
                
                decay = 0.95
                while self._space[key]:
                    self.down += decay
                    decay = decay*decay + 0.1
                    x-=1
                    key = (x, y)
                    
                x = self.id[0]
                y = self.id[1]
                key = (x, y)
                
                decay = 0.95
                while self._space[key]:
                    self.right += decay
                    decay = decay*decay + 0.1
                    y+=1
                    key = (x, y)
                
                x = self.id[0]
                y = self.id[1]
                key = (x, y)
                
                decay = 0.95
                while self._space[key]:
                    self.left += decay
                    decay = decay*decay + 0.1
                    y-=1
                    key = (x, y)
                    
                x = self.id[0]
                y = self.id[1]
                key = (x, y)
                
                decay = 0.95
                while self._space[key]:
                    self.d1 += decay
                    decay = decay*decay + 0.1
                    y-=1
                    x+=1  
                    key = (x, y)
                    
                x = self.id[0]
                y = self.id[1]
                key = (x, y)
                
                decay = 0.95
                while self._space[key]:
                    self.d3 += decay
                    decay = decay*decay + 0.1
                    y+=1
                    x-=1
                    key = (x, y)
                    
                x = self.id[0]
                y = self.id[1]
                key = (x, y)
                
                decay = 0.95
                while self._space[key]:
                    self.d2 += decay
                    decay = decay*decay + 0.1
                    x+=1
                    y+=1   
                    key = (x, y)
                    
                x = self.id[0]
                y = self.id[1]
                key = (x, y)
                
                decay = 0.95
                while self._space[key]:
                    self.d4 +=decay
                    decay = decay*decay + 0.1
                    x-=1
                    y-=1 
                    key = (x, y)
                    
                self.all = self.up + self.down + self.left + self.right 
                + self.d1 + self.d2 + self.d3 + self.d4 
                    
                return self
                    
            def calculate_radius(self, point):
                x_s = self.id[0]
                y_s = self.id[1]
                x_p = point[0]
                y_p = point[1]
                up = self.up + 1 - (x_p - x_s)
                left = self.left + y_p - y_s
                right = self.right + 1 - (y_p - y_s)
                down = self.down + (x_p - x_s)
                
               
                d1 = np.sqrt(2)*(self.d1 + 1)
                d2 = np.sqrt(2)*(self.d2 + 1)
                d3 = np.sqrt(2)*(self.d3 + 1)
                d4 = np.sqrt(2)*(self.d4 + 1)
                
                max_radius = min(up ,left, right, down, d1, d2, d3, d4)
                
                corrected_radius,feedback = self._radius_sweep(max_radius, (x_p, y_p))
                
                return corrected_radius, feedback
            
            def _radius_sweep(self, max_radius, point):
                xs = self.id[0]
                ys = self.id[0]
                x0 = point[0]
                y0 = point[1]
                alpha = 0
                radius = max_radius
                
                while alpha < 360:
                    xn = int(m.floor(x0 + np.sin(alpha)*radius))
                    yn = int(m.floor(y0 + np.cos(alpha)*radius))
                    
                    decay = 0.95
                    feedback = [0, 0]
                    dif_x = np.abs(xn - xs)
                    dif_y = np.abs(yn - ys)
                    
                    for i in range(0, dif_x):
                        for j in range(0, dif_y):
                            key = (xs+i, ys+j)
                            if not (self._space[key]):
                                alpha = 0
                                radius = radius*decay
                                fedback[0]-=1
                                feedback[1]-=1
                                break
                            
                            key = (xs-i, ys-j)
                            if not (self._space[key]):
                                alpha = 0
                                radius = radius*decay
                                fedback[0]+=1
                                feedback[1]+=1
                                break
                                
                            key = (xs+i, ys-j)
                            if not (self._space[key]):
                                alpha = 0
                                radius = radius*decay
                                fedback[0]-=1
                                feedback[1]+=1
                                break
                                
                            key = (xs-i, ys+j)
                            if not (self._space[key]):
                                alpha = 0
                                radius = radius*decay
                                fedback[0]+=1
                                feedback[1]-=1
                                break
                                
                            key = (xs+i, ys)
                            if not (self._space[key]):
                                alpha = 0
                                radius = radius*decay
                                fedback[0]-=1

                                break
                            key = (xs-i, ys)
                            if not (self._space[key]):
                                alpha = 0
                                radius = radius*decay
                                fedback[0]+=1

                                break
                            key = (xs, ys+j)
                            if not (self._space[key]):
                                alpha = 0
                                radius = radius*decay
                                feedback[1]-=1
                                break
                            key = (xs, ys-j)
                            if not (self._space[key]):
                                alpha = 0
                                radius = radius*decay
                                feedback[1]+=1
                                break
                    
                    
                    alpha+=self.angle
                    
                return radius, feedback  
            def candidates(self, npts):
                xs = self.id[0]
                ys = self.id[0]
                points = np.random.uniform(low=0.0, high=1.0, size=(npts, 2))
                final_points = []
                for point in points:
                    final_points.append((point[0]+xs, point[1]+ys))
                return final_points
                    
                
    return



def find_landing(surf_dict, obj_dict):
    N = 30

    npts = 40
    lr = 0.05
    


    
    idx = 0
    points = {}
    for key in obj_dict:
        if not obj_dict[key] is None:

            out_points = obj_dict[key].candidates(npts)
            for point in out_points:
                idx+=1
                points[idx] = point

    for M in range(0, N):

        test = []
        
        for key in obj_dict:
            if surf_dict[key]:
                test.append(obj_dict[key])
        
        test = sorted(test, key = lambda node: node.all)
     
        max_r = -1
        for key in points:
            point = points[key]
            x = int(m.floor(point[0]))
            y = int(m.floor(point[1]))
            new_key = (x, y)
            
            
        
            if not obj_dict[new_key] is None:
               
                r, fb = obj_dict[new_key].calculate_radius(point)
                points[key] = (points[key][0]- lr*fb[1], points[key][1]- lr*fb[0])
                if r > max_r:
                    max_r = r
                

    
        r = max_r

    return (r**2)*np.pi
    

max_r = -1
for area in all_areas:
    
    if not area:
        continue
    

    slice_type = area[1][0]
    delta = area[1][2]
    var = area[1][1]
    
    if slice_type == 'x':
        di = dy +extra
        dj = dz +extra

        surf = np.array(space[var, 0:-1, 0:-1])
        surf_ceil = np.array(space[var+delta, 0:-1, 0:-1])
    if slice_type =='y':
        di = dx +extra
        dj = dz +extra

        surf = np.array(space[0:-1, var, 0:-1])
        surf_ceil = np.array(space[0:-1, var+delta, 0:-1])
    if slice_type == 'z':
        di = dx +extra
        dj = dy +extra
        
        surf = np.array(space[0:-1, 0:-1, var])
        surf_ceil = np.array(space[0:-1, 0:-1, var+delta])

    
    surf_dict = {}
    for i in range(0, di):
        for j in range(0, dj):
            key = (i, j)
            surf_dict[key] = surf[i, j] and not surf_ceil[i, j]

    create(surf_dict)
    obj_dict = {}
    for i in range(0, di):
        for j in range(0, dj):
            key = (i, j)
            if surf_dict[key]:
                obj_dict[key] = Surface((i, j))
            else: 
                obj_dict[key] = None


    for i in range(0, di):
        for j in range(0, dj):
            key = (i, j)

            if not obj_dict[key] is None:
                obj_dict[key].find_surroudings()
    
    
    
    
    r = find_landing(surf_dict, obj_dict)

    if r > max_r:
        max_r = r
    

max_r = round(max_r, 2)
all_areas = sorted(all_areas, key = lambda node:node[0])
side = all_areas[-1][0]

print(f"{total_area} {side} {max_r}")  

0
0
/
|-?
0
0
/
|-?
0
0
/
|-?
0
0
/
|-?
0
0
/
|-?
0
0
/
|-?
0
0
/
|-?
0
0
/
|-?
0
0
/
|-?
0
0
/
|-?


In [342]:
import numpy as np 
import math as m

line = entries[0]
coordinates = []
for el in line:
    a = tuple(int(x) for x in str(el).strip('()\n').split(','))
    coordinates.append(a)


def find_min_max(coordinates):
    x_arr = []
    y_arr = []
    z_arr = []
    
    for el in coordinates:
        x_arr.append(el[0])
        y_arr.append(el[1])
        z_arr.append(el[2])
    
    return ((min(x_arr), max(x_arr)),(min(y_arr), max(y_arr)),(min(z_arr), max(z_arr)))

bounds = find_min_max(coordinates)



dx, dy, dz = bounds[0][1] - bounds[0][0], bounds[1][1] - bounds[1][0], bounds[2][1] - bounds[2][0]

extra = 8
space = np.zeros((dx+extra+1, dy + 1 + extra, dz + extra +1))


for cube in coordinates:

    space[cube[0]-bounds[0][0] + 1:cube[0]+1-bounds[0][0] +1, 
          cube[1]-bounds[1][0] + 1:cube[1]-bounds[1][0]+1 +1,
          cube[2]-bounds[2][0] + 1:cube[2] -bounds[2][0]+1 + 1] = 1


def area(x, y, z, space):
    up =  space[x + 1, y, z] + space[x , y+1, z] + space[x , y, z+1] 
    down = space[x , y - 1, z] + space[x , y, z - 1] + space[x-1 , y, z]
    return 6 - up - down

total_area = 0
for x in range(0, dx+extra ):
    for y in range(0, dy +extra):
        for z in range(0, dz+extra):
            if space[x, y, z] :
                total_area += area(x, y, z, space)


def canVisit(space, row, col, visited, layer):
    global ROW, COL
    
    return ((row >= 0) and (row < ROW) and (col >= 0) and (col < COL) 
           and (space[row][col] and not visited[row][col])) and not layer[row][col]



def DFS(space, row, col, visited, count, layer):
    rowN = [-1, 0, 1, 0]
    colN = [0, -1, 0, 1]
    
    visited[row][col] = True
    
    for k in range(4):
        if(canVisit(space, row + rowN[k],
                     col + colN[k], visited, layer)):
            count[0] += 1
            DFS(space, row + rowN[k],
                     col + colN[k],visited, count, layer)
            
def largestRegion(space, layer):
    global ROW, COL
    
    visited = [[0] * COL for i in range(ROW)]
    ver = []
    result = -1
    
    for i in range(ROW):
        for j in range(COL):
 
            if (space[i][j] and not visited[i][j] and not layer[i][j]):
                 
                
                count = [1]
                DFS(space, i, j, visited, count, layer)

                if(count[0] > result):

                    result = count[0]

    return result
    
x_up = 0
x_down = 0
y_down = 0
y_up = 0
z_up = 0
z_down = 0

ROW = dx +extra
COL = dy +extra

max_top = -1
max_bot = -1
for z in range(0, dz +extra):
    top_layer = space[0:-1, 0:-1, z + 1]
    current_layer = space[0:-1, 0:-1, z]
    bottom_layer =  space[0:-1, 0:-1, z - 1]
    
    top_area = largestRegion(current_layer, top_layer)
    bottom_area = largestRegion(current_layer, bottom_layer)
    
    if top_area > max_top:
        max_top = top_area
        z_up = (top_area, ('z', z, 1))
    if bottom_area > max_bot:
        max_bot = bottom_area
        z_down = (bottom_area, ('z', z, -1))
    

ROW = dy +extra
COL = dz +extra
max_top = -1
max_bot = -1

for x in range(0, dx +extra):
    top_layer = space[x+1, 0:-1, 0:-1]
    current_layer = space[x, 0:-1, 0:-1]
    bottom_layer =  space[x-1, 0:-1, 0:-1]
    
    top_area= largestRegion(current_layer, top_layer)
    bottom_area = largestRegion(current_layer, bottom_layer)
    
    if top_area > max_top:
        max_top = top_area
        x_up = (top_area, ('x', x, 1))
    if bottom_area > max_bot:
        max_bot = bottom_area
        x_down = (bottom_area, ('x', x, -1))
        
ROW = dx +extra
COL = dz +extra

max_top = -1
max_bot = -1
for y in range(0, dy +extra):
    top_layer = space[0:-1, y+1, 0:-1]
    current_layer = space[0:-1, y, 0:-1]
    bottom_layer =  space[0:-1, y-1, 0:-1]
    
    top_area = largestRegion(current_layer, top_layer)
    bottom_area = largestRegion(current_layer, bottom_layer)
    
    if top_area > max_top:
        max_top = top_area
        y_up = (top_area, ('y', y, 1))
    if bottom_area > max_bot:
        max_bot = bottom_area
        y_down = (bottom_area, ('y', y, -1))

all_areas = [x_up, x_down, y_up, y_down, z_up, z_down]



def create(space):
    
    global Surface


    class Surface(object):
            _space = space #dictionary


            def __hash__(self):
                return hash(self.id)

            def __init__(self, coordinates):
                self.id = coordinates


                self.up = 0
                self.down = 0
                self.left = 0
                self.right = 0
                self.d1 = 0
                self.d2 = 0
                self.d3 = 0
                self.d4 = 0
                
                self.angle = 10


            def __repr__(self):
                return str(self.id)

            def find_surroudings(self):
                x = self.id[0]
                y = self.id[1]
                key = (x, y)
                
                decay = 0.95
                while self._space[key]:
                    self.up += decay
                    decay = decay*decay + 0.1
                    x+=1
                    key = (x, y)
                
                x = self.id[0]
                y = self.id[1]
                key = (x, y)
                
                decay = 0.95
                while self._space[key]:
                    self.down += decay
                    decay = decay*decay + 0.1
                    x-=1
                    key = (x, y)
                    
                x = self.id[0]
                y = self.id[1]
                key = (x, y)
                
                decay = 0.95
                while self._space[key]:
                    self.right += decay
                    decay = decay*decay + 0.1
                    y+=1
                    key = (x, y)
                
                x = self.id[0]
                y = self.id[1]
                key = (x, y)
                
                decay = 0.95
                while self._space[key]:
                    self.left += decay
                    decay = decay*decay + 0.1
                    y-=1
                    key = (x, y)
                    
                x = self.id[0]
                y = self.id[1]
                key = (x, y)
                
                decay = 0.95
                while self._space[key]:
                    self.d1 += decay
                    decay = decay*decay + 0.1
                    y-=1
                    x+=1  
                    key = (x, y)
                    
                x = self.id[0]
                y = self.id[1]
                key = (x, y)
                
                decay = 0.95
                while self._space[key]:
                    self.d3 += decay
                    decay = decay*decay + 0.1
                    y+=1
                    x-=1
                    key = (x, y)
                    
                x = self.id[0]
                y = self.id[1]
                key = (x, y)
                
                decay = 0.95
                while self._space[key]:
                    self.d2 += decay
                    decay = decay*decay + 0.1
                    x+=1
                    y+=1   
                    key = (x, y)
                    
                x = self.id[0]
                y = self.id[1]
                key = (x, y)
                
                decay = 0.95
                while self._space[key]:
                    self.d4 +=decay
                    decay = decay*decay + 0.1
                    x-=1
                    y-=1 
                    key = (x, y)
                    
                self.all = self.up + self.down + self.left + self.right 
                + self.d1 + self.d2 + self.d3 + self.d4 
                    
                return self
                    
            def calculate_radius(self, point):
                x_s = self.id[0]
                y_s = self.id[1]
                x_p = point[0]
                y_p = point[1]
                up = self.up + 1 - (x_p - x_s)
                left = self.left + y_p - y_s
                right = self.right + 1 - (y_p - y_s)
                down = self.down + (x_p - x_s)
                
               
                d1 = np.sqrt(2)*(self.d1 + 1)
                d2 = np.sqrt(2)*(self.d2 + 1)
                d3 = np.sqrt(2)*(self.d3 + 1)
                d4 = np.sqrt(2)*(self.d4 + 1)
                
                max_radius = min(up ,left, right, down, d1, d2, d3, d4)
                
                corrected_radius,feedback = self._radius_sweep(max_radius, (x_p, y_p))
                
                return corrected_radius, feedback
            

            def candidates(self, npts):
                xs = self.id[0]
                ys = self.id[0]
                points = np.random.uniform(low=0.0, high=1.0, size=(npts, 2))
                final_points = []
                for point in points:
                    final_points.append((point[0]+xs, point[1]+ys))
                return [(xs + 0.5, ys+0.5)]
            
            
            def _find_optimal_radius(self, point, lr, decay, growth, max_iter):
                
                growth2 = 1000*growth
                
                x_p = point[0]
                y_p = point[1]
                
                x_s = self.id[0]
                y_s = self.id[0]
                #print(x_s, y_s)
                
                r = 0.1
                iter_count = 1
                proj = np.sqrt(2)/2
                flag = False
                
                max_r = -1
                
                comb_x = {}
                comb_y = {}
                
                
                while iter_count < max_iter:
                    alpha = 0
                    while alpha < 360 :
                        comb_x[alpha] = r*np.sin(alpha)
                        comb_y[alpha] = r*np.cos(alpha)
                        alpha+=self.angle
                        
                    
                    
                    for _, dx in comb_x.items():
                        for _, dy in comb_y.items():
                            k1 = m.floor(x_p + dx)
                            k2 = m.floor(y_p + dy)
                            
                            if k1 < 0 or k2<0:
                                
                                iter_count += 1
                                
                                x_p = x_p - lr*np.sign(dx)
                                y_p = y_p - lr*np.sign(dy)
                                

                                if k1<0 :
                                    k1 = 0
                                if k2<= 0:
                                    k2 = 0
                            
                                
                                
                                
                                
                           
                                
                                
                            key = (k1, k2)
                            
                            #print(key)
                            
                            #print(self._space[key])
                            
                            if not self._space[key]:
                            
                                x_p = x_p - lr*np.sign(dx)
                               
                                y_p = y_p - lr*np.sign(dy)
                                
                                #print(x_p, y_p, r)
                                

                                
                                
                                
                                
                                flag = True

                                iter_count += 1
                                #print(x_p, y_p, r)
                                
                                
                        else:
                            continue
                        break
                    
                    
                    
                        
                    if not flag:
                        if r > max_r:
                            max_r = r
                        r = r + growth + growth2
                        growth2 = growth2*decay
                        flag = False
                    else:
                        r = r*decay
                        flag = False
                    
                
        
                return max_r
            
            def competition(self, npts, lr, decay, growth, max_iter):
                
                points = self.candidates(npts)
                #print(points)
                max_r = -1
                for point in points:
                    r = self._find_optimal_radius(point, lr, decay, growth, max_iter)
                    if r > max_r:
                        max_r = r
                return max_r
                    
                
    return



def find_landing(surf_dict, obj_dict):
    max_iter = 100
    growth = 0.002
    decay = 0.95
    lr = 0.0002
    npts = 3

    
    test = []
        
    for key in obj_dict:
        if surf_dict[key]:
            test.append(obj_dict[key])
        
    test = sorted(test, key = lambda node: node.all)
    
    #print(test[-1].all)
        
    max_r = -1
    for i in range(3):
        if i + 1 > len(test):
            break
        r =test[-1-i].competition(npts, lr, decay, growth, max_iter)
        if r > max_r:
            max_r = r

   

    return (max_r**2)*np.pi
    

max_r = -1
for area in all_areas:
    
    if not area:
        continue
    

    slice_type = area[1][0]
    delta = area[1][2]
    var = area[1][1]
    
    if slice_type == 'x':
        di = dy +extra
        dj = dz +extra

        surf = np.array(space[var, 0:-1, 0:-1])
        surf_ceil = np.array(space[var+delta, 0:-1, 0:-1])
    if slice_type =='y':
        di = dx +extra
        dj = dz +extra

        surf = np.array(space[0:-1, var, 0:-1])
        surf_ceil = np.array(space[0:-1, var+delta, 0:-1])
    if slice_type == 'z':
        di = dx +extra
        dj = dy +extra
        
        surf = np.array(space[0:-1, 0:-1, var])
        surf_ceil = np.array(space[0:-1, 0:-1, var+delta])

    
    surf_dict = {}
    for i in range(0, di):
        for j in range(0, dj):
            key = (i, j)
            surf_dict[key] = surf[i, j] and not surf_ceil[i, j]

    create(surf_dict)
    obj_dict = {}
    for i in range(0, di):
        for j in range(0, dj):
            key = (i, j)
            if surf_dict[key]:
                obj_dict[key] = Surface((i, j))
            else: 
                obj_dict[key] = None


    for i in range(0, di):
        for j in range(0, dj):
            key = (i, j)

            if not obj_dict[key] is None:
                obj_dict[key].find_surroudings()
    
    
    
    
    r = find_landing(surf_dict, obj_dict)

    if r > max_r:
        max_r = r
     

max_r = round(max_r, 2)
all_areas = sorted(all_areas, key = lambda node:node[0])
side = all_areas[-1][0]

print(f"{total_area} {side} {max_r}")

KeyError: (12, 11)